<a href="https://colab.research.google.com/github/MithunCm29/MachineLearning/blob/main/HaiX_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
import nltk
nltk.download('stopwords')
!pip install fasttext
import pandas as pd
import re
import fasttext
!pip install spacy
from nltk.corpus import stopwords
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
stop = stopwords.words('english')
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 68 kB 4.2 MB/s 
  Using cached pybind11-2.7.0-py2.py3-none-any.whl (199 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093415 sha256=331eceff0acf37ea4579aa8c7971846374c221133875fd73a4c9c1f63234211e
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  del sys.path[0]


In [4]:
#Data Reading
data = pd.read_csv("/content/gdrive/MyDrive/HaixData/labelled data  - Mithun_Ishita_mindvalley_TO DO.csv",encoding= 'unicode_escape')
test= pd.read_csv("/content/gdrive/MyDrive/HaixData/labelled data  - mindvalley_distilbert_16-07.csv",encoding= 'unicode_escape')
df = pd.read_csv("/content/gdrive/MyDrive/HaixData/labelled data  - Testing Data.csv",encoding= 'unicode_escape')

In [5]:
def process_text(mystr):
        text = str(mystr)
        #clearstring = re.sub('[^a-zA-Z0-9 \n\.]', Replacement, mystr)
        text = text.lower()
        text = re.sub("@[a-z0-9_-]+","", text)        #Removing @mentions
        text = re.sub("#[a-z0-9-_]+", "", text)       #Removing #Hashtags
        text = re.sub(r"http\S+", "", text)
        text = re.sub("[^a-z]", " ", text)
        text = re.sub("\n","", text)
        text = re.sub("^\[a-z0-9]+","", text)
        text = nlp(text)
        text = [tok.lemma_ for tok in text]
        text = ' '.join(text)        
        return text

In [8]:
#Data Process for training
data['Processed_Text'] = [process_text(text) for text in data['tweet']]
data['Consolidated.1'] = [label.upper() for label in data['Consolidated.1']]
data["Final"] = "__label__" + data['Consolidated.1'] +" "+ data['Processed_Text']

In [9]:
#Data Process for Testing
test['Processed_Text'] = [process_text(text) for text in test['tweet']]
test['sentiment_distilbert'] = [label.upper() for label in test['sentiment_distilbert']]
test["Final"] = "__label__" + test['sentiment_distilbert'] +" "+ test['Processed_Text']

In [10]:
#Data Process For predicting
df['text'] = [process_text(text) for text in df['tweet']]

In [11]:
with open('/content/sentiment_fasttext_train_New_All.txt', 'w',encoding="utf-8") as f:
    for item in data["Final"]:
        f.write("%s\n" % item)
with open('/content/sentiment_fasttext_test_New_All.txt', 'w',encoding="utf-8") as f:
    for item in test["Final"]:
        f.write("%s\n" % item)

In [12]:
#Model Train
model = fasttext.train_supervised(input="/content/sentiment_fasttext_train_New_All.txt",lr=0.1, epoch=10, bucket=2000000, dim=300, loss='softmax', wordNgrams=2, autotuneValidationFile="/content/sentiment_fasttext_train_New_All.txt")

In [13]:
#Model Test
model.test("/content/sentiment_fasttext_test_New_All.txt")

(178, 0.6460674157303371, 0.6460674157303371)

In [14]:
#Model Predict
df['predictedsentiment'] = [model.predict(txt)[0][0].split("__")[2] for txt in df['text']]
print(df['predictedsentiment'])


0        NEUTRAL
1        NEUTRAL
2        NEUTRAL
3        NEUTRAL
4        NEUTRAL
          ...   
21228    NEUTRAL
21229    NEUTRAL
21230    NEUTRAL
21231    NEUTRAL
21232    NEUTRAL
Name: predictedsentiment, Length: 21233, dtype: object
